In [1]:
COLAB_DRIVE_DIR = "/content/drive/MyDrive/Soil types"

In [2]:
# Required Imports
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


In [3]:
# Image Shape and batch size (same as notebook)
IMG_SIZE = (160, 160)
IMG_SHAPE = IMG_SIZE + (3,)
BATCH_SIZE = 32

In [4]:
# Data Augmentation + Preprocessing (Exact steps)
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [5]:
#Data Generators (same split logic)
train_generator = train_datagen.flow_from_directory(
    COLAB_DRIVE_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset='training',
    shuffle=True,
    class_mode='categorical',
    seed=42
)

Found 126 images belonging to 5 classes.


In [6]:
val_generator = train_datagen.flow_from_directory(
    COLAB_DRIVE_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset='validation',
    shuffle=False,
    class_mode='categorical',
    seed=42
)

Found 30 images belonging to 5 classes.


In [7]:
# 9️⃣ Class labels (same mapping logic)
CLASS_NAMES = list(train_generator.class_indices.keys())
print("Detected Classes:", CLASS_NAMES)

Detected Classes: ['Black Soil', 'Cinder Soil', 'Laterite Soil', 'Peat Soil', 'Yellow Soil']


In [9]:
base_model = tf.keras.applications.EfficientNetB4(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)

# Freeze first 100 layers and fine-tune rest (same strategy)
for layer in base_model.layers[:100]:
    layer.trainable = False

In [10]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
outputs = Dense(len(CLASS_NAMES), activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb4 (Functional)     │ (None, 5, 5, 1792)     │    17,673,823 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1792)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1792)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │         8,965 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,682,788 (67.45 MB)

 Trainable params: 17,470,821 (66.65 MB)

 Non-trainable params: 211,967 (828.00 KB)

In [11]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
early_stop = EarlyStopping(patience=5, restore_best_weights=True)
lr_reduce = ReduceLROnPlateau(patience=2, factor=0.5, min_lr=1e-6)
checkpoint = ModelCheckpoint("efficientnetb4_soil.h5", save_best_only=True)

In [13]:
EPOCHS = 20
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stop, lr_reduce, checkpoint]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.2761 - loss: 1.6616 

4/4 ━━━━━━━━━━━━━━━━━━━━ 170s 20s/step - accuracy: 0.2733 - loss: 1.6583 - val_accuracy: 0.3667 - val_loss: 1.5799 - learning_rate: 1.0000e-04
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.3137 - loss: 1.5410

4/4 ━━━━━━━━━━━━━━━━━━━━ 38s 9s/step - accuracy: 0.3224 - loss: 1.5282 - val_accuracy: 0.4333 - val_loss: 1.4999 - learning_rate: 1.0000e-04
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.5623 - loss: 1.3028

4/4 ━━━━━━━━━━━━━━━━━━━━ 36s 9s/step - accuracy: 0.5578 - loss: 1.3028 - val_accuracy: 0.4000 - val_loss: 1.4951 - learning_rate: 1.0000e-04
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 36s 9s/step - accuracy: 0.6623 - loss: 1.1748 - val_accuracy: 0.4000 - val_loss: 1.5184 - learning_rate: 1.0000e-04
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.7456 - loss: 1.0231

...

4/4 ━━━━━━━━━━━━━━━━━━━━ 36s 9s/step - accuracy: 0.9068 - loss: 0.3955 - val_accuracy: 0.7667 - val_loss: 0.9337 - learning_rate: 1.0000e-04
Epoch 14/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.9549 - loss: 0.3210

4/4 ━━━━━━━━━━━━━━━━━━━━ 36s 9s/step - accuracy: 0.9560 - loss: 0.3198 - val_accuracy: 0.8333 - val_loss: 0.8294 - learning_rate: 1.0000e-04
Epoch 15/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9346 - loss: 0.2880

4/4 ━━━━━━━━━━━━━━━━━━━━ 37s 9s/step - accuracy: 0.9350 - loss: 0.2894 - val_accuracy: 0.8000 - val_loss: 0.7402 - learning_rate: 1.0000e-04
Epoch 16/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 37s 10s/step - accuracy: 0.9703 - loss: 0.2317 - val_accuracy: 0.7333 - val_loss: 0.8375 - learning_rate: 1.0000e-04
Epoch 17/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.9619 - loss: 0.1957

4/4 ━━━━━━━━━━━━━━━━━━━━ 39s 10s/step - accuracy: 0.9632 - loss: 0.1959 - val_accuracy: 0.7667 - val_loss: 0.7142 - learning_rate: 1.0000e-04
Epoch 18/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.9399 - loss: 0.2004

4/4 ━━━━━━━━━━━━━━━━━━━━ 39s 10s/step - accuracy: 0.9440 - loss: 0.1995 - val_accuracy: 0.8000 - val_loss: 0.6900 - learning_rate: 1.0000e-04
Epoch 19/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 37s 9s/step - accuracy: 0.9707 - loss: 0.1760 - val_accuracy: 0.7667 - val_loss: 0.7294 - learning_rate: 1.0000e-04
Epoch 20/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 37s 9s/step - accuracy: 0.9422 - loss: 0.1985 - val_accuracy: 0.7333 - val_loss: 0.7430 - learning_rate: 1.0000e-04


In [14]:
model.save("/content/drive/MyDrive/soil_efficientnetb4.h5")
print("✅ Training Completed and Model Saved!")

✅ Training Completed and Model Saved!


In [15]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [16]:
def predict_soil(img_path):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    img_batch = preprocess_input(img_batch)

    preds = model.predict(img_batch)[0]
    class_idx = np.argmax(preds)
    confidence = preds[class_idx]

    return CLASS_NAMES[class_idx], confidence

In [20]:
sample_image = "/content/drive/MyDrive/Soil types/Peat Soil/20.jpg"
soil, conf = predict_soil(sample_image)
print(f"Predicted Soil: {soil}, Confidence: {conf:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
Predicted Soil: Peat Soil, Confidence: 0.65
